In [1]:
import os
import sys
import numpy as np
import cv2
from datetime import datetime

import torch

In [2]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, img_x, img_y, transform=None):
        self.df = df
        self.data_dir = data_dir
        self.img_x = img_x
        self.img_y = img_y
        self.transform = transform
        lst_label = list(df['label'])
        lst_input = list(x for x in df.image_id.values)
        self.lst_label = lst_label
        self.lst_input = lst_input


    def __len__(self):
        return len(self.lst_label)


    def __getitem__(self, index):
        label = self.lst_label[index]
        input = cv2.imread(os.path.join(self.data_dir, self.lst_input[index]), cv2.IMREAD_COLOR)
        input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)  # result of input shape is y,x,c

        if self.transform:
            input = self.transform(image=input)['image']

        data = {'input' : input, 'label' : label}

        return data

In [3]:
##
import os
import sys
import numpy as np
import pandas as pd

# pytorch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.cuda.amp.autocast_mode import autocast
from torch.cuda.amp import GradScaler
from torchcontrib.optim import SWA

# sci-kit learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import albumentations as A
from albumentations.pytorch import ToTensorV2


In [4]:
def transform_train(img_x, img_y):
    return A.Compose([A.RandomResizedCrop(img_x, img_y),
                      A.Transpose(p=0.5),
                      A.HorizontalFlip(p=0.5),
                      A.VerticalFlip(p=0.5),
                      A.ShiftScaleRotate(p=0.5),
                      A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                      A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                      A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                      A.CoarseDropout(p=0.5), A.Cutout(p=0.5),ToTensorV2(p=1.0)], p=1.)


def transform_val(img_x, img_y):
    return A.Compose([A.CenterCrop(img_x, img_y, p=1.),
                      A.Resize(img_x, img_y),
                      A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),ToTensorV2(p=1.0)], p=1.)


In [5]:
import timm

import torch
import torch.nn as nn

##
class CassvaImgClassifier(nn.Module):
    def __init__(self, network, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(network, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)

        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''

    def forward(self, x):
        x = self.model(x)
        return x

In [6]:

def dataloader_train(df, data_dir, trn_idx, val_idx, img_x, img_y, batch_size):
    train = df.loc[trn_idx, :].reset_index(drop=True)
    val = df.loc[val_idx, :].reset_index(drop=True)

    dataset_train = TrainDataset(df=train, data_dir=data_dir, img_x=img_x, img_y=img_y, transform=transform_train(img_x, img_y))
    loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=8)

    dataset_val = TrainDataset(df=val, data_dir=data_dir, img_x=img_x, img_y=img_y, transform=transform_val(img_x, img_y))
    loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers=8)

    num_data_train = len(dataset_train)
    num_data_val = len(dataset_val)

    num_batch_train = np.ceil(num_data_train / batch_size)
    num_batch_val = np.ceil(num_data_val / batch_size)

    return loader_train, loader_val, num_batch_train, num_batch_val


In [7]:
def train(args):
    # hyperparameters
    train_continue = args.train_continue
    seed = args.seed

    img_x = args.img_x
    img_y = args.img_y

    lr = args.lr
    num_fold = args.num_fold
    num_epoch = args.num_epoch
    batch_size = args.batch_size
    
    data_dir = args.data_dir
    ckpt_dir = args.ckpt_dir

    network = args.network

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    ##
    df = pd.read_csv('./data/train.csv')
    Kfold = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=seed)

    for fold, (trn_idx, val_idx) in enumerate(Kfold.split(np.arange(df.shape[0]), df.label.values)):
        if fold > 0:
            break

        print("Training start ... ")
        print("DATE: {} | ".format(datetime.now().strftime("%m.%d-%H:%M")), "KFOLD: {}/{}".format(fold, num_fold))

        loader_train, loader_val, num_batch_train, num_batch_val = \
            dataloader_train(df, data_dir, trn_idx, val_idx, img_x, img_y, batch_size)

        net = CassvaImgClassifier(network, df.label.nunique(), pretrained=True).to(device)
        fn_loss = nn.CrossEntropyLoss().to(device) 
        optim = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-6)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
        scaler = GradScaler()

        ##
        st_epoch = 0
        best_loss = 1e20
        best_acc = 0
        early_stop_patience = 0

        if train_continue == "on":
            net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

        train_loss, val_loss, train_acc, val_acc = [], [], [], []

        for epoch in range(st_epoch + 1, num_epoch + 1):
            net.train()
            loss_arr = []  # loss array for one batch

            # loss/accuracy for one epoch
            loss_epoch_train = 0
            acc_epoch_train = 0

            # cuda device로 보내는 부분의 용량을 줄이기 위해서 long과 float을 해줫더니 배치도 늘어나고 쿠다도 들어감
            
            for batch, data in enumerate(loader_train, 1):
                # forward pass
                label = data['label'].to(device).long()
                input = data['input'].to(device).float()
                

                with autocast():
                    output = net(input)
                    print(output)
                    break
                    loss = fn_loss(output, label)
                    #print(loss)
                    optim.zero_grad()

                # backward pass
                scaler.scale(loss).backward()
                scaler.step(optim)
                scaler.update()




                # loss.backward()
                # optim.step()

                loss_arr += [loss.item()]
                loss_batch_train = np.mean(loss_arr)
                loss_epoch_train += loss_batch_train

                output = torch.argmax(output, dim=1).cpu().detach().numpy()
                label = label.cpu().detach().numpy()
                acc_batch_train = accuracy_score(label, output)
                acc_epoch_train += acc_batch_train

            train_loss.append(loss_epoch_train / num_batch_train)
            train_acc.append(acc_epoch_train / num_batch_train)

            
            scheduler.step(train_loss[-1])

            with torch.no_grad():
                net.eval()
                loss_arr = []

                loss_epoch_val = 0
                acc_epoch_val = 0

                for batch, data in enumerate(loader_val, 1):
                    label = data['label'].to(device).long()
                    input = data['input'].to(device).float()

                    output = net(input)

                    loss = fn_loss(output, label)

                    loss_arr += [loss.item()]
                    loss_batch_val = np.mean(loss_arr)
                    loss_epoch_val += loss_batch_val

                    output = torch.argmax(output, dim=1).cpu().detach().numpy()
                    label = label.cpu().detach().numpy()
                    acc_batch_val = accuracy_score(label, output)
                    acc_epoch_val += acc_batch_val

                val_loss.append(loss_epoch_val / num_batch_val)
                val_acc.append(acc_epoch_val / num_batch_val)

            scheduler.step(val_loss[-1])

            print("DATE: {} | ".format(datetime.now().strftime("%m.%d-%H:%M")), "EPOCH: {}/{} | ".format(epoch, num_epoch),
                  "TRAIN_LOSS: {:4f} | ".format(train_loss[-1]),  "TRAIN_ACC: {:4f} | ".format(train_acc[-1]),
                  "VAL_LOSS: {:4f} | ".format(val_loss[-1]), "VAL_ACC: {:4f} | ".format(val_acc[-1]))

In [8]:
## 라이브러리 추가하기
import os
import numpy as np
import random

import torch

import argparse

## Parser 생성하기
parser = argparse.ArgumentParser(description="Cassava Leaf Disease Classification",
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--mode", default="train", choices=["train", "test"], type=str, dest="mode")
parser.add_argument("--train_continue", default="off", choices=["on", "off"], type=str, dest="train_continue")

parser.add_argument("--seed", default=719, type=int, dest="seed")
parser.add_argument("--img_x", default=512, type=int, dest="img_x")
parser.add_argument("--img_y", default=512, type=int, dest="img_y")

parser.add_argument("--lr", default=1e-4, type=float, dest="lr")
parser.add_argument("--num_fold", default=5, type=int, dest="num_fold")
parser.add_argument("--num_epoch", default=10, type=int, dest="num_epoch")
parser.add_argument("--batch_size", default=8, type=int, dest="batch_size")

parser.add_argument("--data_dir", default="./data/train_images", type=str, dest="data_dir")
parser.add_argument("--ckpt_dir", default="./checkpoint", type=str, dest="ckpt_dir")
parser.add_argument("--result_dir", default="./result", type=str, dest="result_dir")

parser.add_argument("--network", default="tf_efficientnet_b4_ns", type=str, dest="network")
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args, unknown = parser.parse_known_args()

##
if __name__ == "__main__":
    # random seed
    def seed_everything(seed):
        random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

    seed = args.seed
    seed_everything(seed)

    if args.mode == "train":
        train(args)
    elif args.mode == "test":
        test(args)

# tensorboard --logdir ./log/train


Training start ... 
DATE: 01.15-17:32 |  KFOLD: 0/5
tensor([[-0.1736, -0.0427,  0.0158,  0.0220,  0.0210],
        [-0.1326,  0.0092,  0.0056, -0.0325,  0.0194],
        [-0.0288, -0.0897, -0.0828, -0.1086,  0.1046],
        [ 0.0036, -0.0721, -0.0434, -0.0005, -0.1387],
        [ 0.1370,  0.0578,  0.0423,  0.1295,  0.1976],
        [ 0.1209,  0.0155,  0.0259,  0.0798,  0.0057],
        [-0.0374,  0.0362, -0.0069, -0.0933, -0.0832],
        [-0.1527, -0.0215, -0.0057, -0.0333, -0.1107]], device='cuda:0',
       dtype=torch.float16, grad_fn=<AddmmBackward>)


KeyboardInterrupt: 